# Pipeline Tool


When creating a domain tool from scratch, it is convenient to leverage existing chains and tools. Below are two examples that demonstrate the integration of DuckDuckSearch and a Fake LLM Chain to accomplish a domain task.


## Complete Output Result using Pipeline Tool

The provided demo pipeline tool comprises three steps:

1. Retrieving the result from DuckDuckSearch
2. Converting the output into a dictionary format using ``yaml.safe_load`` with single-quotes treated as JSON-like syntax.
3. Generating a summary of the output by selecting specific fields.

In [2]:
import json
import yaml

from langchain.llms.fake import FakeListLLM
from langchain import PromptTemplate
from langchain.chains import MapReduceDocumentsChain, LLMChain

from langchain.tools import DuckDuckGoSearchResults


/home/andy/miniconda3/envs/lc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
responses = [
    """Here is the first summary for duckduck ...""", 
    """Here is the second summary for duckduck ..."""
            ]
llm = FakeListLLM(responses=responses)
prompt_template = "Summarize {documents} to me"
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))


In [5]:
dd = DuckDuckGoSearchResults()

In [6]:
import yaml
from langchain.tools import PipelineStep, PipelineTool

ppl = PipelineTool(
    name='duckduck_search_summarization', 
    description='useful when duckduck search summarization is required',
    steps=[
        PipelineStep(func=dd, output_expression='dd_output'),
        PipelineStep(func=lambda x: yaml.safe_load(x), input_expression={'x': 'dd_output'}, output_expression={'documents': '[*].{title: title, summary: snippet}'}),
        PipelineStep(func=llm_chain, input_expression={'inputs': 'documents'}, output_expression={'summary': 'text'}),
    ]
)



In [7]:
output = ppl('duckduck search')

In [8]:
doc = json.loads(output)

In [9]:
doc

{'dd_output': '[{\'snippet\': "By contrast, DuckDuckGo search is completely anonymous and if you add our app & extension on top, we help keep you private when browsing off of search results. Learn more. How popular is DuckDuckGo? We don\'t track our users, so it\'s actually impossible for us to know how many users in total are using our products. However, we are able to ...", \'title\': \'DuckDuckGo - Privacy, simplified.\', \'link\': \'https://duckduckgo.com/\'}, {\'snippet\': "DuckDuckGo released a companion web browser for Mac in public beta in 2022, and now there\'s a Windows version available. DuckDuckGo\'s browser is now available in beta for Windows computers, letting you try out the browser and all of its private, secure features.", \'title\': \'DuckDuckGo Web Browser Now Available for Windows PCs\', \'link\': \'https://www.howtogeek.com/900963/duckduckgo-web-browser-now-available-for-windows-pcs/\'}, {\'snippet\': "Windows users have a new way to browse the web with more priva

In [12]:
doc['summary']

'Here is the summary for duckduck ...'

## Pipeline Tool with Final Output


By utilizing the ``output_expression`` argument as a list containing a single string element, the pipeline tool will return the final result extracted from the output using the specified key..

In [10]:
import yaml
from langchain.tools import PipelineStep, PipelineTool

ppl = PipelineTool(
    name='duckduck_search_summarization', 
    description='useful when duckduck search summarization is required',
    steps=[
        PipelineStep(func=dd, output_expression='dd_output'),
        PipelineStep(func=lambda x: yaml.safe_load(x), input_expression={'x': 'dd_output'}, output_expression={'documents': '[*].{title: title, summary: snippet}'}),
        PipelineStep(func=llm_chain, input_expression={'inputs': 'documents'}, output_expression={'summary': 'text'}),
    ],
    output_expression=['summary'],
)



In [11]:
ppl('duckduck search')

'"Here is the second summary for duckduck ..."'